# 字典和集合

字典在 Python 中至关重要，即使没有直接使用，也会间接用到，因为 <mark>dict 类型是实现 Python 的基石</mark>

> 一些 Python 核心结构在内存中以字典的形式存在，比如类和实例属性、模块命名空间，以及函数的关键字

> 由于字典很重要，所以 Python 对字典做了高度优化，而且一直在改进，Python 字典高效的原因是使用了哈希表

> 除了字典之外，`set` 和 `frozenset` 也是基于哈希表，而且这两个类型的运算符和 API 相比于其他语言的集合更加丰富
>
> <span style="color:green"> Python 中的集合实现了集合论中的所有基本运算，包括并集、交集、子集测试等</span>

包括以下内容：
- 构建及处理 dict 和<mark>映射</mark>的现代语法, 包括增强的拆包和模式匹配
- 映射类型的常用方法
- 特别处理缺失键的情况
- 标准库中的 dict 变体
- set 和 frozenset 类型
- 哈希表对集合和字典行为的影响

> dict 和 set 的底层实现依赖于哈希表，不过 dict 的代码有两项重要的优化，节省内存，还能保留键的插入顺序

## 3.2 字典的现代句法

### 字典推导式

In [1]:
dial_codes = [
    (880, 'Bangladesh'),
    (55, 'Brazil'),
    (86, 'China'),
    (91, 'India'),
    (81, 'Japan'),
]

country_dial = {country: code for code, country in dial_codes}
print(country_dial)

{'Bangladesh': 880, 'Brazil': 55, 'China': 86, 'India': 91, 'Japan': 81}


In [2]:
{code: country.upper() 
 for country, code in sorted(country_dial.items())
 if code < 100
}

{55: 'BRAZIL', 86: 'CHINA', 91: 'INDIA', 81: 'JAPAN'}

### 映射拆包

Python3.5 开始对映射拆包功能做了增强

- 调用函数时，不止一个参数可以使用 `**`, 但是<mark>所有的键都要是字符串，而且在所有的参数重是唯一的</mark>
- `**` 可以在 dict 字面量中使用，同样可以多次使用

In [3]:
def dump(**kwargs):
    return kwargs

dump(**{'x': 1}, y=2, **{'z': 3, 't': 4})

{'x': 1, 'y': 2, 'z': 3, 't': 4}

In [1]:
{'a': 0, **{'x': 1}, 'y': 2, **{'z': 3, 'x': 4}}

{'a': 0, 'x': 4, 'y': 2, 'z': 3}

### 使用 | 合并映射

In [4]:
d1 = dict(a=1, b=2)
d2 = dict(c=3, d=4, a=6)

print(d1 | d2)

{'a': 6, 'b': 2, 'c': 3, 'd': 4}


更新后的 a 是 6

通常来说，新映射的类型与左操作数的类型相同

可以通过就地修改操作，使用 `|=` 更改当前现有的映射

## 3.3 使用模式匹配处理映射

`match/case` 语句可以匹配映射。

不同类型的模式可以组合和嵌套，模式匹配是一种强大的工具，借助析构可以处理嵌套的映射和序列等结构化记录

> 经常需要从 JSON API 和具有半结构化模式的数据库（MongoDB、PG）读取数据

In [6]:
# 从出版物记录中提取创作者的名字

def get_creators(record: dict) -> list:
    match record:
        case {'type': 'book', 'api': 2, 'authors': [*names]}:
            return names
        case {'type': 'book', 'api': 1, 'author': name}:
            return [name]
        case {'type': 'book'}:
            raise ValueError(f"Invalid 'book' record: {record!r}")
        case {'type': 'movie', 'director': name}:
            return [name]
        case _:
            raise ValueError(f"Invalid record: {record!r}")

下面做一个小小的测试，测试一下 get_creators 函数

In [10]:
b1 = dict(api=1, author='Douglas Adams', type='book', title='war of the worlds')

author = get_creators(b1)
print(author)

from collections import OrderedDict
b2 = OrderedDict(api=2, type='book', title='war of the worlds', authors='Douglas Adams'.split())
authors = get_creators(b2)
print(authors)

get_creators({'type': 'book', 'pages': 100})

['Douglas Adams']
['Douglas', 'Adams']


ValueError: Invalid 'book' record: {'type': 'book', 'pages': 100}

<mark>模式中的键的顺序无关紧要</mark>

<span style="color: green"> 与序列模式不同，就算只有部分匹配，映射模式也算匹配成功 </span>


<span style="color:green"> 没有必要使用 `**extra` 匹配多出的键值对，若想把多余的键值对捕获到一个 dict 中，可以在一个变量前面加上 `**`, 不过需要放在最后面 </span> 

In [11]:
food = dict(category='fruit', name='apple', price=1.99)
match food:
    case {'category': 'fruit', **details}:
        print(f'fruit details: {details}')

fruit details: {'name': 'apple', 'price': 1.99}


> 模式匹配不自动处理缺失的键，因为模式匹配始终使用 `d.get(key, sentinel)` 方法，其中，sentinel 是特殊的标记，不会出现在用户数据中。

## 3.4 映射类型的标准 API